<a href="https://colab.research.google.com/github/hariskhan-hk/Tumor_Classification/blob/main/Tumor_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tumor Classification

# I. Import Required Dependecies

In [2]:
import os
import glob
from IPython.display import Image as IPImage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image           # Pillow - for image processing tasks

# TensorFlow - for deep learning operations and model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

# Keras - for building and training neural network models
from keras.layers import Flatten, Dense, Activation, Dropout
from keras import models, optimizers
from keras.models import Sequential
from keras.constraints import MaxNorm
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import DenseNet121
from keras.applications.densenet import DenseNet121, preprocess_input

# scikit-learn - for data splitting and model evaluation utilities
from sklearn.model_selection import train_test_split